In [1]:
import pandas as pd
import numpy as np
import sklearn
import json
import time

In [2]:
# print("Printed immediately.")
# time.sleep(2.4)
# print("Printed after 2.4 seconds.")

In [3]:
# with open('data.txt', 'w') as outfile:
#     json.dump(x, outfile)

In [4]:
# with open('data.txt') as json_file:
#     data = json.load(json_file)

In [5]:
# for i in range(len(dates_from)):
#     data = search(dates_from[i], dates_to[i], 5)
#     with open(f'data/data{i}.txt', 'w') as outfile:
#         json.dump(data, outfile)
#     time.sleep(5)

In [6]:
# def job_id_extract(data):
#     list = []
#     for i in data['result']:
#         list.append(i['job_id'])
        
#     return(list)

In [ ]:
# my_dict = {}
# for i in range(len(dates_from)):
#     with open(f'data/data{i}.txt') as json_file:
#         my_dict[i] = json.load(json_file)

In [19]:
dates_from = ['2020-09-28 00:00', '2020-09-29 00:00', '2020-9-30 00:00', '2020-10-01 00:00', '2020-10-02 00:00', '2020-10-03 00:00', '2020-10-04 00:00', '2020-10-05 00:00', '2020-10-06 00:00', '2020-10-07 00:00', '2020-10-08 00:00', '2020-10-09 00:00', '2020-10-10 00:00', '2020-10-11 00:00', '2020-10-12 00:00', '2020-10-13 00:00', '2020-10-14 00:00', '2020-10-15 00:00', '2020-10-16 00:00', '2020-10-17 00:00', '2020-10-18 00:00', '2020-10-19 00:00', '2020-10-20 00:00']
dates_to = ['2020-09-29 00:00', '2020-09-30 00:00', '2020-10-01 00:00', '2020-10-02 00:00', '2020-10-03 00:00', '2020-10-04 00:00', '2020-10-05 00:00', '2020-10-06 00:00', '2020-10-07 00:00', '2020-10-08 00:00', '2020-10-09 00:00', '2020-10-10 00:00', '2020-10-11 00:00', '2020-10-12 00:00', '2020-10-13 00:00', '2020-10-14 00:00', '2020-10-15 00:00', '2020-10-16 00:00', '2020-10-17 00:00', '2020-10-18 00:00', '2020-10-19 00:00', '2020-10-20 00:00', '2020-10-21 00:00']

# Introduction, Business Understanding

Questions:
- How does Hybrid Analysis construct its threat score for URL's?
    - Can a generalized model be made based on the reports their Falcon Sandbox platform produces?
- What features are important to Hybrid Analysis URL threat score?
    - Can these features be determined, and then in turn, easily be extracted with guidance from their reports.
    - If reverse engineering isn't possible, can generealized insights still be produced?
    
Objectives:
- Creae a model that interprets Hybrid Analysis reports to determine threat score on URL's
- Create a feature list of Hybrid Analysis most informative features
- If the above are successful, deploy a reverse engineered model

Success Criteria:
- Either a functional model or insights into Hybrid Analysis feature importance for URL's that could guide another project

Hybrid Analysis:
- a website that allows users to submit malware, url's executables, html, etc for a proprietary "threat score" analysis.

# What have I done so far?

made search query and writing to file functions
made job_id extracting function that takes a dictionary input

can I get the things I need?
I need
- large set of malicious URL's
    - I can get these using my search and write function using a large enough window
- the malicious URLS's accompanying JOB id's
    - I can get these by extraccting the job ID's using my function on the dictionary
- Now I have job id's, target values, now I need features.
    - I need to use report get to get reports for each of my malicios URLS's I am storing that have the features
- Researching what is exactly contained in the feature reports
    - MUTEX (Mutual Exclusion) Locking down parts of code by keeping it in use, sometimes bloking visibility of other programs and making it immutable to outside other programs, think "file is in use somewhere else, cannot delete"

In [9]:
from search_query import search
from report_get import report
from job_id_extract import job_id_extract
from search_write_to_file import search_and_write
from dict_make import dict_make
from report_get_write_to_file import report_write

In [2]:
# dates_from = ['2020-09-28 00:00', '2020-09-29 00:00', '2020-9-30 00:00', '2020-10-01 00:00', '2020-10-02 00:00', '2020-10-03 00:00', '2020-10-04 00:00', '2020-10-05 00:00', '2020-10-06 00:00', '2020-10-07 00:00', '2020-10-08 00:00', '2020-10-09 00:00', '2020-10-10 00:00', '2020-10-11 00:00', '2020-10-12 00:00', '2020-10-13 00:00', '2020-10-14 00:00', '2020-10-15 00:00', '2020-10-16 00:00', '2020-10-17 00:00', '2020-10-18 00:00', '2020-10-19 00:00', '2020-10-20 00:00','2020-10-21 00:00', '2020-10-22 00:00', '2020-10-23 00:00', '2020-10-24 00:00', '2020-10-25 00:00', '2020-10-26 00:00', '2020-10-27 00:00', '2020-10-28 00:00','2020-10-29 00:00', '2020-11-01 00:00', '2020-11-02 00:00', '2020-11-03 00:00', '2020-11-04 00:00', '2020-11-05 00:00', '2020-11-06 00:00', '2020-11-07 00:00', '2020-11-08 00:00', '2020-11-09 00:00', '2020-11-10 00:00', '2020-11-11 00:00', '2020-11-12 00:00', '2020-11-13 00:00', '2020-11-14 00:00', '2020-11-15 00:00', '2020-11-16 00:00', '2020-11-17 00:00', '2020-11-18 00:00', '2020-11-19 00:00', '2020-11-20 00:00', '2020-11-21 00:00', '2020-11-22 00:00', '2020-11-23 00:00', '2020-11-24 00:00', '2020-11-25 00:00']
# dates_to = ['2020-09-29 00:00', '2020-09-30 00:00', '2020-10-01 00:00', '2020-10-02 00:00', '2020-10-03 00:00', '2020-10-04 00:00', '2020-10-05 00:00', '2020-10-06 00:00', '2020-10-07 00:00', '2020-10-08 00:00', '2020-10-09 00:00', '2020-10-10 00:00', '2020-10-11 00:00', '2020-10-12 00:00', '2020-10-13 00:00', '2020-10-14 00:00', '2020-10-15 00:00', '2020-10-16 00:00', '2020-10-17 00:00', '2020-10-18 00:00', '2020-10-19 00:00', '2020-10-20 00:00', '2020-10-21 00:00', '2020-10-22 00:00', '2020-10-23 00:00', '2020-10-24 00:00', '2020-10-25 00:00', '2020-10-26 00:00', '2020-10-27 00:00', '2020-10-28 00:00','2020-10-29 00:00', '2020-11-01 00:00', '2020-11-02 00:00', '2020-11-03 00:00', '2020-11-04 00:00', '2020-11-05 00:00', '2020-11-06 00:00', '2020-11-07 00:00', '2020-11-08 00:00', '2020-11-09 00:00', '2020-11-10 00:00', '2020-11-11 00:00', '2020-11-12 00:00', '2020-11-13 00:00', '2020-11-14 00:00', '2020-11-15 00:00', '2020-11-16 00:00', '2020-11-17 00:00', '2020-11-18 00:00', '2020-11-19 00:00', '2020-11-20 00:00', '2020-11-21 00:00', '2020-11-22 00:00', '2020-11-23 00:00', '2020-11-24 00:00', '2020-11-25 00:00', '2020-11-26 00:00']

In [11]:
len(search('2020-09-28 00:00', '2020-09-29 00:00', 5))

3

In [9]:
# search_and_write(dates_from, dates_to)

In [3]:
with open('data/data0.txt') as json_file:
    data = json.load(json_file)
data

{'search_terms': [{'id': 'filetype', 'value': 'url'},
  {'id': 'env_id', 'value': '100'},
  {'id': 'verdict', 'value': '5'},
  {'id': 'date_from', 'value': '2020-09-28 00:00'},
  {'id': 'date_to', 'value': '2020-09-29 00:00'}],
 'count': 4,
 'result': [{'verdict': 'malicious',
   'av_detect': '2',
   'threat_score': 100,
   'vx_family': 'Malicious site',
   'job_id': '5f72774a50643e7bc7460603',
   'sha256': '18b965df088347aceff0a2e6cfd5abfe9150df9b529d902b0a28125efec25b33',
   'environment_id': 100,
   'analysis_start_time': '2020-09-28 23:52:45',
   'submit_name': 'httpchuyengiadienmay.webdungsan.com.url',
   'environment_description': 'Windows 7 32 bit',
   'size': 63,
   'type': None,
   'type_short': 'url'},
  {'verdict': 'malicious',
   'av_detect': '0',
   'threat_score': 90,
   'vx_family': None,
   'job_id': '5f7274e3e6faf447dd379eca',
   'sha256': 'abd1ab47f8f3e0da5060d9e54c46e2959d9d53d4e13903d97d8c2f1198968ce3',
   'environment_id': 100,
   'analysis_start_time': '2020-09-28

In [16]:
range(len(dates_from))

range(0, 54)

In [21]:
my_dict = dict_make(dates_from)

In [24]:
len(my_dict)

23

In [25]:
my_dict

{0: {'search_terms': [{'id': 'filetype', 'value': 'url'},
   {'id': 'env_id', 'value': '100'},
   {'id': 'verdict', 'value': '5'},
   {'id': 'date_from', 'value': '2020-09-28 00:00'},
   {'id': 'date_to', 'value': '2020-09-29 00:00'}],
  'count': 4,
  'result': [{'verdict': 'malicious',
    'av_detect': '2',
    'threat_score': 100,
    'vx_family': 'Malicious site',
    'job_id': '5f72774a50643e7bc7460603',
    'sha256': '18b965df088347aceff0a2e6cfd5abfe9150df9b529d902b0a28125efec25b33',
    'environment_id': 100,
    'analysis_start_time': '2020-09-28 23:52:45',
    'submit_name': 'httpchuyengiadienmay.webdungsan.com.url',
    'environment_description': 'Windows 7 32 bit',
    'size': 63,
    'type': None,
    'type_short': 'url'},
   {'verdict': 'malicious',
    'av_detect': '0',
    'threat_score': 90,
    'vx_family': None,
    'job_id': '5f7274e3e6faf447dd379eca',
    'sha256': 'abd1ab47f8f3e0da5060d9e54c46e2959d9d53d4e13903d97d8c2f1198968ce3',
    'environment_id': 100,
    'ana

In [66]:
len(range(len(dates_from)))

23

In [22]:
my_list = job_id_extract(my_dict)

In [24]:
with open('reports/reports5f7e47fdb8e47e68de0a152a.txt') as json_file:
    data = json.load(json_file)
data

{'response': {'uuid': 'java:java.util.UUID',
  'xmlns': 'http://www.misp-project.org/',
  'Event': {'id': 'aa4e6f40-f451-4e2e-8d62-e3bb89af86b0',
   'date': '2020-10-07',
   'info': 'Falcon Sandbox auto-generated for "f399f059fe85ad2f58fcfc24f4fd830d8877a768803dfa5dc25ca93b4c673c22.url"',
   'analysis': 2,
   'distribution': 1,
   'published': 1,
   'Attribute': [{'category': 'External analysis',
     'type': 'link',
     'value': 'https://www.hybrid-analysis.com/search?query=f399f059fe85ad2f58fcfc24f4fd830d8877a768803dfa5dc25ca93b4c673c22',
     'distribution': 1},
    {'category': 'External analysis',
     'type': 'comment',
     'value': 'Falcon Sandbox v8.43 Copyright 2019 Hybrid Analysis GmbH, All Rights Reserved, www.payload-security.com',
     'distribution': 1},
    {'category': 'Payload delivery',
     'type': 'filename|md5',
     'value': 'f399f059fe85ad2f58fcfc24f4fd830d8877a768803dfa5dc25ca93b4c673c22.url|40ce0241a6c915bf62bedd89aa73f4c0',
     'distribution': 1},
    {'cat

In [23]:
len(my_list)

159

In [16]:
report_write(my_list)

In [17]:
rep2 = report(my_list[1])

In [18]:
rep2

{'response': {'uuid': 'java:java.util.UUID',
  'xmlns': 'http://www.misp-project.org/',
  'Event': {'id': 'ad1c1758-6eee-41e5-bf32-9469bb03964d',
   'date': '2020-09-28',
   'info': 'Falcon Sandbox auto-generated for "abd1ab47f8f3e0da5060d9e54c46e2959d9d53d4e13903d97d8c2f1198968ce3.url"',
   'analysis': 2,
   'distribution': 1,
   'published': 1,
   'Attribute': [{'category': 'External analysis',
     'type': 'link',
     'value': 'https://www.hybrid-analysis.com/search?query=abd1ab47f8f3e0da5060d9e54c46e2959d9d53d4e13903d97d8c2f1198968ce3',
     'distribution': 1},
    {'category': 'External analysis',
     'type': 'comment',
     'value': 'Falcon Sandbox v8.43 Copyright 2019 Hybrid Analysis GmbH, All Rights Reserved, www.payload-security.com',
     'distribution': 1},
    {'category': 'Payload delivery',
     'type': 'filename|md5',
     'value': 'abd1ab47f8f3e0da5060d9e54c46e2959d9d53d4e13903d97d8c2f1198968ce3.url|3ecdc8b5d1561cdc806e8c6f71a7628a',
     'distribution': 1},
    {'cat

In [16]:
rep = report("5f9a05839fb71b523c1a2b59")

In [17]:
len(rep['response']['Event']['Attribute'])

535

In [59]:
# for i in rep['response']['Event']['Attribute']:
#     print(i)

In [64]:
rep['response']['Event']['Attribute'][2]

{'category': 'Payload delivery',
 'type': 'filename|md5',
 'value': '290605d4e1a27ba1bf5539f0390d4b7437fe0eac0260984cf4c5fa259fc97b11.url|c01e09696b3597b4d57b95fe4a54abc7',
 'distribution': 1}

In [ ]:
159

# features
feature count:payload_delivery

rep['response']['Event']['Attribute'][i] - the i here is arbitrary, but what this does is drills down to the features, the 'attributes' are the features, mutex, payload delivery, persistence mechanism etc. can just count the number of attributes. If you add ['value'] to the end of the statement it gets the values of i which is typically a something with a hash?

avg length of value in payload_delivery

- can just use both on any number of feature possibilities

max length of something/anything

distance between feature_id 1 and 2 != 2 and 3

# Functions

dict_make - makes a combined dictionary of all downloaded data files from using search_and_write

job_id_extract - loops through the dictionary created by dict_make to append the job_id to a list

report_write - takes the job_id list extracted from job_id_extract and queries hybrid_analysis for the report that contains features
report_get - the GET request used in report_write


search_write_to_file - passes the date lists to search_query and gets the returns that contain the threat scores, then saves them as dataX
search_query - the post request used in search_write_to_file